In [1]:
import re
from pyodbc import connect
from deep_translator import GoogleTranslator
from pandas import DataFrame, read_csv
import pandas as pd
import json

In [2]:
# FUNC cleanNumber
def cleanNumber(text):
    s = re.sub('(\(\d+\))', '', text)
    return re.sub('\s+\.\s+', '. ', s).strip()
def cleanFirstNumber(text):
    text = cleanNumber(text)
    return re.sub('^\d+\.\s*', '', text).strip()

In [3]:
# FUNC fetchAll, saveMany of SQL
def fetchAll(sql, parameters = None):
    db = connect('Driver={SQL Server};Server=.;Database=Hospital;UID=sa;PWD=123')
    try:
        cursor = db.cursor()
        if parameters:
            cursor.execute(sql, parameters)
        else:
            cursor.execute(sql)
        return cursor.fetchall()
    except Exception as ex:
        print(ex)
        return False
    finally:
        cursor.close()
        db.close()      
def saveMany(sql, arr):
    db = connect('Driver={SQL Server};Server=.;Database=Hospital;UID=sa;PWD=123')
    try:
        cursor = db.cursor()
        cursor.executemany(sql, arr)
        ret = cursor.rowcount
        db.commit()
        return True
    except Exception as ex:
        print(ex)
        return False
    finally:
        cursor.close()
        db.close()

In [21]:
# FUNC translate_text
def translate_text(text):
    translator = GoogleTranslator(source='vi', target='en')
    try:
        return translator.translate(text)
    except Exception as e:
        return "Error: " + str(e)

In [4]:
# SELECT DATA
sql = '''
select Url,FieldEn,ArticleEn,Item.ItemId, ItemEn, ContentEn from Item 
join Paragraph on Item.ItemId = Paragraph.ItemId 
join Article on Article.ArticleId=Item.ArticleId
join Field on Field.FieldId = Article.FieldId
WHERE IsDeleted = 0
ORDER BY Paragraph.ParagraphId
'''
data = fetchAll(sql)

In [5]:
# GO merge paragraph MERGE THEO MỐC 430 WORDS
def merge_paragraphs(data):
    dataset = []
    d = {'Url': data[0][0], 'FieldEn': data[0][1], 'ArticleEn': data[0][2], 'ItemEn': data[0][4], 'ContentEn': cleanFirstNumber(data[0][-1])}
    cnt = len(cleanFirstNumber(data[0][-1]).split())
    for i in range(1, len(data)):
        length = len(cleanFirstNumber(data[i][-1]).split())
        if data[i][2] == data[i-1][2]:
            if cnt + length < 430:
                d['ContentEn'] += cleanFirstNumber(data[i][-1])
            else:
                dataset.append(d)
                cnt = 0
                d = {'Url': data[i][0], 'FieldEn': data[0][1], 'ArticleEn': data[i][2], 'ItemEn': data[i][4], 'ContentEn': cleanFirstNumber(data[i][-1])}
        else:
            dataset.append(d)
            cnt = 0
            d = {'Url': data[i][0], 'FieldEn': data[0][1], 'ArticleEn': data[i][2], 'ItemEn': data[i][4], 'ContentEn': cleanFirstNumber(data[i][-1])}
        cnt += length
    dataset.append(d)
    return dataset
dataset = merge_paragraphs(data)

In [ ]:
df = DataFrame(dataset)
df['Content_Length'] = df['ContentEn'].str.split().map(len)

In [ ]:
break430 = df
break430['Combined_Content_2'] = break430['ItemEn'].astype(str) + ' ' + break430['ContentEn']
break430['Combined_Length_2'] = break430['Combined_Content_2'].str.split().map(len)
combined_title = break430.drop(columns=['FieldEn', 'ArticleEn', 'ItemEn', 'ContentEn', 'Content_Length'])
combined_title.sort_values('Combined_Length_2',ascending=False)

In [ ]:
combined_title.to_csv('data_break430_231110.csv')